# Hive Assignment 2. DML: Find Most Popular Tags
Compare most popular tags in year 2016 with tags in 2009. Tag popularity is the number of questions (post_type_id=1) with this tag. Output top 10 tags in format:

    tag <tab> rank in 2016 <tab> rank in 2009 <tab> tag popularity in 2016 <tab> tag popularity in 2009

Example:

    php 5 3 1234 5678

For 'rank' in each year use rank() function. Order by 'rank in 2016'.

The part of the result on the sample dataset:

    ...
    android 3   52  1809    25
    php 4   3   1673    215
    python  5   11  1585    108
    c#  6   1   1519    423
    ...

Please use the tables 'posts' and 'users' from the database 'stackoverflow_'. 'posts' is partitioned by year and by month. For more details see "Hive assignment. Intro and instructions".


In [3]:
%%writefile task2_top_tags.hql

SELECT
  l.tag, l.rk, r.rk, l.cnt, r.cnt
FROM
(
  SELECT
  tag,
    cnt,
    ROW_NUMBER() OVER (ORDER BY cnt DESC) AS rk
  FROM (
    SELECT tag, COUNT(tag) AS cnt FROM (
    SELECT year, tag FROM
        posts LATERAL VIEW explode(tags) posts AS tag
    WHERE year = 2016
        AND tags IS NOT NULL
        AND post_type_id = 1
    ) tmp2016
    GROUP BY tag
) count2016
) l
     JOIN
(
  SELECT
  tag,
    cnt,
    ROW_NUMBER() OVER (ORDER BY cnt DESC) AS rk
  FROM (
    SELECT tag, COUNT(tag) AS cnt FROM (
    SELECT year, tag FROM
        posts LATERAL VIEW explode(tags) posts AS tag
    WHERE year = 2009
        AND tags IS NOT NULL
        AND post_type_id = 1
    ) tmp2009
    GROUP BY tag
) count2009
) r
ON l.tag = r.tag
ORDER BY l.rk
LIMIT 10
;



Overwriting task2_top_tags.hql


In [4]:
! hive --database stackoverflow_ -f task2_top_tags.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.128 seconds
Query ID = jovyan_20190727194444_73b79554-d462-439b-aa8f-8077a23e2ee8
Total jobs = 8
Launching Job 1 out of 8
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1564233419780_0006, Tracking URL = http://66698698fae4:8088/proxy/application_1564233419780_0006/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1564233419780_0006
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2019-07-27 19:45:07,402 Stage-1 map = 0%,  reduce = 0%
2019-07-27 19:45:17,315 Stage-1 map = 100%, 